<a href="https://colab.research.google.com/github/CamiloVga/Curso-Inteligencia-Artificial/blob/main/Clase_Fine_Tuning_ChatBot_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning ChatBot LLama 2

Explicación paso a paso para crear un modelo Fine-Tuning de un modelo ChatBot por medio de un modelo Llama 2 de generación de texto. Se incluye la opción para hacerlo tanto con datos sintéticos como para hacerlo con datos propios.

El modelo está Quantanizado a 8 bits

**Creado por [Camilo Vega](https://www.linkedin.com/in/camilo-vega-169084b1/)**

# Datos de entrenamiento

In [ ]:
#Opción 1: Datos sintéticos

!pip install openai==0.28

import openai
import pandas as pd

# Paso 2: Configurar la clave de API de OpenAI
openai.api_key = 'API KEY'

# Paso 3: Definir variables para la generación de texto
num_ejemplos = 100  # Número de ejemplos de texto a generar
max_tokens = 200  # Longitud máxima de la respuesta

# Paso 4: Generar la base de datos con la API de OpenAI
instrucciones = []
salidas = []
for _ in range(num_ejemplos):
    prompt = "Dame una instrucción sobre generación de noticias de 50 palabras sobre inteligencia artificial"
    mensajes = [{"role": "user", "content": prompt}]
    respuesta_instruccion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=mensajes,
        max_tokens=50,
        temperature=0.7,
        n=1,
    )
    instruccion = respuesta_instruccion["choices"][0]["message"]["content"]
    instrucciones.append(instruccion)

    mensajes = [{"role": "user", "content": f"Instrucción: {instruccion}\nSalida:"}]
    respuesta_salida = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=mensajes,
        max_tokens=max_tokens,
        temperature=0.7,
        n=1,
    )
    salida = respuesta_salida["choices"][0]["message"]["content"]
    salidas.append(salida)

# Paso 5: Crear el DataFrame con las instrucciones y salidas
df = pd.DataFrame({"instruction": instrucciones, "output": salidas})

# Paso 6: Guardar el DataFrame en un archivo CSV
df.to_csv("train.csv", index=False)

# Paso 7: Mostrar el DataFrame resultante
df

## Opción 2: Datos propios

También se puede usar un data set con datos propios. Es cuestión de seguir los siguientes pasos:

1. Abrir una hoja sheet en Drive
2. Crear un columna llamada "instruction" y otra "output"
3. En la columna "instruction" poner los ejemplos de instrucciones que se le daría a un asistente
4. En la columna "output" poner la respectiva respuesta que aplique para cada instrucción.
5. Guardar el archivo en formato .csv. Se recomienda guardarlo bajo el nombre "train" (train.csv) para que el siguiente código aplique a los datos propios.
6. Subir los datos a Colab. La ruta es ir al símbolo de la carpeta, elegir la opción de subir archivo (el símbolo de la hoja con la flecha) y buscar la base de datos en el computador.

In [2]:
#Convertir la base de datos al formato de Llama
import pandas as pd

def convert_to_custom_format(input_file, output_file):
    # Leer el dataset original
    dataset = pd.read_csv(input_file)

    # Crear una nueva lista para almacenar los textos formateados
    formatted_texts = []

    # Iterar sobre cada fila del dataset
    for index, row in dataset.iterrows():
        # Construir el texto formateado con las instrucciones y las salidas
        formatted_text = f"<s>[INST] {row['instruction']} [/INST] {row['output']} </s>"

        # Agregar el texto formateado a la lista
        formatted_texts.append(formatted_text)

    # Crear un nuevo DataFrame con una sola columna llamada "text"
    new_dataset = pd.DataFrame({"text": formatted_texts})

    # Guardar el nuevo dataset en un archivo CSV
    new_dataset.to_csv(output_file, index=False)

# Llamar a la función con el nombre del archivo de entrada y de salida
convert_to_custom_format("train.csv", "custom_train.csv")

# Cargar el archivo custom_train.csv en un DataFrame
custom_df = pd.read_csv("custom_train.csv")

# Mostrar las primeras filas del DataFrame
custom_df

,text
0,<s>[INST] La generación de noticias sobre inte...
1,"<s>[INST] ""La inteligencia artificial está rev..."
2,<s>[INST] La noticia sobre el avance de la int...
3,<s>[INST] La generación de noticias sobre inte...
4,<s>[INST] La inteligencia artificial está revo...
...,...
95,<s>[INST] La generación de noticias sobre inte...
96,<s>[INST] La inteligencia artificial está revo...
97,"<s>[INST] ""La inteligencia artificial está rev..."
98,<s>[INST] La generación de noticias sobre inte...


# Entrenamiento

In [3]:
#Instalar librerias
!pip install -q accelerate==0.21.0 peft==0.4.0 transformers==4.31.0 trl==0.4.7 pandas datasets
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00


In [4]:
#Configuración de los hiperparametros

import os
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# El modelo que deseas entrenar desde el Hugging Face Hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Nombre del archivo CSV en Colab
dataset_file = "custom_train.csv"

# Nombre del modelo Fine-Tuning
new_model = "llama2-chatbot"

# Cargar conjunto de datos desde el archivo CSV en Colab
dataset = pd.read_csv(dataset_file)

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 8-bit precision base model loading
load_in_8bit = True

# Disable 4-bit quantization
llm_int8 = False
lm_int8 = False
output_in_8bit = False
threshold_for_int8 = 6.0

# No need for compute dtype or quantization type
# bnb_4bit_compute_dtype = "float16"
# bnb_4bit_quant_type = "nf4"

# Deactivate nested quantization
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


In [5]:
#Entrenamiento

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8=False,
    lm_int8=False,
    output_in_8bit=False,
    threshold_for_int8=6.0,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar conjunto de datos desde el archivo CSV en Colab
dataset_df = pd.read_csv(dataset_file)

# Dividir el DataFrame en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(dataset_df, test_size=0.2, random_state=42)

# Convertir los DataFrames a objetos Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Configurar el entrenador SFTTrainer con el conjunto de entrenamiento
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


# Probar el modelo


In [6]:
# Inferencia de generación de texto
logging.set_verbosity(logging.CRITICAL)

prompt = "Escribe una noticia de 50 palabras sobre la IA"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Escribe una noticia de 50 palabras sobre la IA [/INST]  La IA ha revolucionado la forma en que interactuamos con la tecnología. Ahora, nuestros dispositivos pueden aprender de nosotros y adaptarse a nuestras necesidades, mejorando la eficiencia y la experiencia de usuario.


In [7]:
#Inferencia por medio de modalidad Chatbot
import logging
from transformers import pipeline

# Configurar el nivel de registro a CRITICAL
logging.basicConfig(level=logging.CRITICAL)

# Crear el pipeline para generación de texto
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

# Mensaje de inicio
print("Chatbot: Hola, soy un chatbot. ¿En qué puedo ayudarte?")

# Bucle para responder preguntas del usuario hasta que el usuario dice "terminar"
while True:
    # Solicitar al usuario que escriba una pregunta o instrucción
    user_input = input("Usuario: ")

    # Verificar si el usuario quiere terminar la conversación
    if user_input.lower() == "terminar":
        print("Chatbot: Hasta luego.")
        break  # Salir del bucle si el usuario dice "terminar"

    # Generar una respuesta basada en la entrada del usuario y mostrarla
    result = pipe(f"[INST] {user_input} [/INST]")
    print("Chatbot:", result[0]['generated_text'])
    print("Chatbot: ¿Ahora en qué puedo ayudarte? Si quieres cerrar sesión escribe: terminar")

Chatbot: Hola, soy un chatbot. ¿En qué puedo ayudarte?
Usuario: qué es la inteligencia artificial?
Chatbot: [INST] qué es la inteligencia artificial? [/INST]  La inteligencia artificial (IA) es un campo de la informática que se enfoca en el desarrollo de sistemas capaces de realizar tareas que requieren inteligencia humana, como el aprendizaje, el razonamiento, el reconocimiento de voz y el reconocimiento de imágenes.

La IA utiliza algoritmos y modelos matemáticos para analizar grandes cantidades de datos y tomar decisiones basadas en ellos. Algunos ejemplos de aplicaciones de la IA son:

1. Sistemas de recomendación: como los que sugieren películas o productos en línea basados en el historial de compras o el comportamiento del usuario.
2. Chatbots: son programas de computadora que pueden interactuar con los usuarios de manera conversacional, como asistentes virtuales o sistemas de respuesta a preguntas.
3. Vehículos autónomos: son coches que pueden conducirse sin la intervención huma

# Publicar el modelo

In [ ]:
#HuggingFaceLogin
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#PushModeltoHub
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)